In [1]:
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping
from keras.losses import binary_crossentropy
from tensorflow.keras import regularizers
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
height, width, channels = 372, 1244, 1  #input image format
input_img = Input(shape=(height, width, channels))
#print(input_img)

In [3]:
# Encoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

In [4]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(learning_rate = 0.001), loss='mse')
autoencoder.summary()

def preprocess_image(image):
    #resize image so they're same format
    image = tf.image.resize(image, (height, width), method='lanczos5')
    return image

datagen = ImageDataGenerator(
    #normalize image so values are 0>= and <=1
    rescale=1./255,
    preprocessing_function=preprocess_image
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 372, 1244, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 372, 1244, 16)  │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 186, 622, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 186, 622, 8)    │         1,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 93, 311, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 93, 311, 8)     │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 47, 156, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 47, 156, 8)     │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 94, 312, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 94, 312, 8)     │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 188, 624, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 186, 622, 16)   │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 372, 1244, 16)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 372, 1244, 1)   │           145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
import os
os.getcwd()

'/Users/yong/Desktop'

In [5]:
data_generator = datagen.flow_from_directory(
    'thesis/dataset',  #path to dataset
    target_size=(height, width),
    color_mode='grayscale',
    batch_size=16,
    class_mode='input'
)

Found 80 images belonging to 5 classes.


In [6]:
#train model
autoencoder.fit(data_generator, epochs=15)

Epoch 1/15


/Users/yong/anaconda3/envs/plard/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - loss: 0.2431
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - loss: 0.2397
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.2333
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.2210
Epoch 5/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.1982
Epoch 6/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.1628
Epoch 7/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - loss: 0.1163
Epoch 8/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - loss: 0.0574
Epoch 9/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.0133
Epoch 10/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.0031
Epoch 11/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - loss: 0.0024
Epoch 12/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.0022
Epoch 13/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - loss: 0.0021
Epoch 14/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 0.0020
Epoch 15/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.0013


In [7]:
#save model
autoencoder.save('0428.keras')